In [ ]:
'''匯入套件'''
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json, os, time, re
from pprint import pprint
from urllib import parse

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent(cache=True) # cache=True 表示從已經儲存的列表中提取

'''設定 Chrome 瀏覽器開啟時的狀態'''
my_options = webdriver.ChromeOptions()
my_options.add_argument('--start-maximized')
my_options.add_argument('--incognito')
my_options.add_argument('--disable-popup-blocking')
my_options.add_argument(f'--user-agent={ua.random}')

'''建立操控 Chrome 瀏覽器的變數'''
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

'''放置 金庸小說 metadata 的資訊'''
listData = []

'''小庸小說的網址'''
url = 'https://www.bookwormzz.com/zh/'

# 沒有放置 txt 檔的資料夾，就建立起來
folderPath = 'jinyong'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
# 取得小說的主要連結
def getMainLinks():
    # 走訪首頁
    driver.get(url)
    
    # 取得主要連結
    a_elms = driver.find_elements(By.CSS_SELECTOR, 'a[data-ajax="false"]')
    
    # 整理主要連結資訊
    for a in a_elms:
        listData.append({
            "title": a.get_attribute('innerText'),
            "link": parse.unquote( a.get_attribute('href') ) + "#book_toc",
            "sub": [] # 為了放置各個章回小說的內頁資料，下一個步驟會用到
        })

# 取得所有章回小說的連結
def getSubLinks():
    for i in range( len(listData) ):
        # 走訪章回小說內頁
        driver.get(listData[i]["link"])
        
        # 若是走訪網頁時，等待不到特定的元素，視為沒有資料，continue 到 for 的下一個 index 去
        try:
            # 等待元素
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'div[data-theme="b"][data-content-theme="c"] a[rel="external"]')
                )
            )
            
            # 整理章回小說
            a_elms = driver.find_elements(By.CSS_SELECTOR, 'div[data-theme="b"][data-content-theme="c"] a[rel="external"]')
            for a in a_elms:
                listData[i]["sub"].append({
                    "sub_title": a.get_attribute("innerText"),
                    "sub_link": parse.unquote( a.get_attribute("href") )
                })
        except TimeoutException as e:
            continue

# 建立金庸小說的 json 檔
def saveJson():
    with open(f"{folderPath}/jinyong.json", "w", encoding="utf-8") as file:
        file.write( json.dumps(listData, ensure_ascii=False) )

# 將金庸小說所有章回的內容，各自寫到 txt 與 json 中
def writeTxt():  
    # 稍候建立 train.json 前的程式變數
    listContent = []

    # 開啟 金庸小說 metadata 的 json 檔
    with open(f"{folderPath}/jinyong.json", "r", encoding="utf-8") as file:
        strJson = file.read()

    # 走訪所有章回的小說文字內容
    listResult = json.loads(strJson)
    for i in range( len(listResult) ):
        for j in range( len(listResult[i]["sub"]) ):
            # 走訪內頁
            driver.get( listResult[i]['sub'][j]['sub_link'] )
            div = driver.find_element(By.CSS_SELECTOR, 'div#html > div')
            
            # 取得內文
            strContent = div.get_attribute('innerText')
            
            # 資料預處理
            strContent = re.sub(r" |\r|\n|　|\s", '', strContent)

            # 決定 txt 的檔案名稱
            fileName = f"{listResult[i]['title']}_{listResult[i]['sub'][j]['sub_title']}.txt"

            # 將小說內容存到 txt 中
            with open(f"{folderPath}/{fileName}", "w", encoding="utf-8") as file:
                file.write( strContent )

            # 額外將小說內容放到 list 當中，建立 train.json
            listContent.append(strContent)

    # 延伸之後的教學，在此建立訓練資料
    with open(f"{folderPath}/train.json", "w", encoding="utf-8") as file:
        file.write( json.dumps(listContent, ensure_ascii=False) )

# 關閉瀏覽器
def close():
    driver.quit()

In [ ]:
# 主程式
if __name__ == "__main__":
    time1 = time.time()
    getMainLinks()
    getSubLinks()
    saveJson()
    writeTxt()
    close()
    print(f"執行總花費時間: {time.time() - time1}")